In [ ]:
!pip install transformers accelerate torch bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
MODEL_ID = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def analyze_code(code_snippet):
    prompt = f"""Analyze the following Python code, find any errors, and provide a corrected version:

    {code_snippet}

    **Response Format:**
    - **Faulty Line:** #<line_number>
    - **Issue:** <explanation>
    - **Fixed Code:**
    ```python
    <corrected_code>
    ```
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=500)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Test the function
code_snippet = """
#1 def calculate_average(nums):
#2     total = sum(nums)
#3     return total / len(num)
#4 nums=[1,2]
#5 calculate_average(nums)
"""

print(analyze_code(code_snippet))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Analyze the following Python code, find any errors, and provide a corrected version:

    
#1 def calculate_average(nums):
#2     total = sum(nums)
#3     return total / len(num)
#4 nums=[1,2]
#5 calculate_average(nums)


    **Response Format:**
    - **Faulty Line:** #<line_number>
    - **Issue:** <explanation>
    - **Fixed Code:**
    ```python
    <corrected_code>
    ```
    
    **Corrected Code:**
    - **Faulty Line:** #2
    - **Issue:** The function `calculate_average` is not defined.
    - **Fixed Code:**
    ```python
    def calculate_average(nums):
        total = sum(nums)
        return total / len(nums)
    nums=[1,2]
    calculate_average(nums)
    ```
    
    **Corrected Code:**
    - **Faulty Line:** #3
    - **Issue:** The variable `len` is not defined.
    - **Fixed Code:**
    ```python
    def calculate_average(nums):
        total = sum(nums)
        return total / len(nums)
    nums=[1,2]
    calculate_average(nums)
    ```
    
    **Corrected Code:**
    

In [ ]:
def analyze_code(code_snippet):
    prompt = f"""Analyze the following Python code, find any errors, and provide a corrected version:

    {code_snippet}

    **Response Format:**
    - **Faulty Line:** #<line_number>
    - **Issue:** <explanation>
    - **Fixed Code:**
    ```python
    <corrected_code>
    ```
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=600)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Test the function
code_snippet = """
#1 def fibonacci(n):
#2   fib_seq = [0, 1]
#3   for i in range(2, n):
#4      fib_seq.append(fib_seq[i-1] + fib_seq[i-2])
#5   return fib_seq[:n]
#6
#7 n = input("Enter the number of terms")
#8 print(fibonacci(n))
"""

print(analyze_code(code_snippet))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Analyze the following Python code, find any errors, and provide a corrected version:

    
#1 def fibonacci(n):
#2   fib_seq = [0, 1]
#3   for i in range(2, n):
#4      fib_seq.append(fib_seq[i-1] + fib_seq[i-2]) 
#5   return fib_seq[:n]
#6
#7 n = input("Enter the number of terms") 
#8 print(fibonacci(n))


    **Response Format:**
    - **Faulty Line:** #<line_number>
    - **Issue:** <explanation>
    - **Fixed Code:**
    ```python
    <corrected_code>
    ```
    
    **Corrected Code:**
    - **Faulty Line:** #2
    - **Issue:** The variable `fib_seq` is not defined.
    - **Fixed Code:**
    ```python
    #1 def fibonacci(n):
    #2   fib_seq = [0, 1]
    #3   for i in range(2, n):
    #4      fib_seq.append(fib_seq[i-1] + fib_seq[i-2]) 
    #5   return fib_seq[:n]
    #6
    #7 n = input("Enter the number of terms") 
    #8 print(fibonacci(n))
    ```

    - **Faulty Line:** #4
    - **Issue:** The `fib_seq` list is not defined.
    - **Fixed Code:**
    ```python
    #1 def fib